In [2]:
import open3d as o3d
import numpy as np
import math
import glob
import os


In [18]:
# pcd = o3d.io.read_point_cloud("/home/aichi2204/ros2_ws/test/pcl-test/data/map_all_removed/best2/39.pcd")

# 対象フォルダのパスを指定
folder_path = '/home/aichi2204/Documents/bkl2go/20240412-library2/aichi-20240412-library2_croped_mini_1/1'  # ここを対象フォルダのパスに置き換えてください
# パターンにマッチするファイルをすべて取得
file_pattern = os.path.join(folder_path, '[0-9]*.pcd')
file_paths = glob.glob(file_pattern)
pcds = []
for file_path in file_paths:
    pcd = o3d.io.read_point_cloud(file_path)
    pcds.append(pcd)

In [19]:
# 合成
pcd_combined = pcds[0]
for pcd in pcds[1:]:
    pcd_combined += pcd
print(pcd_combined)

# スケーリング
# print(pcd_combined.get_max_bound() , pcd_combined.get_min_bound())
# scale_= 1 / np.max(pcd_combined.get_max_bound() - pcd_combined.get_min_bound())
# center_=pcd_combined.get_center()
# pcd_combined.scale(scale_,center=center_)
# for i in range(len(pcds)):
#     pcds[i].scale(scale_,center=center_)


PointCloud with 6125025 points.


In [22]:
print(pcd_combined.get_max_bound() - pcd_combined.get_min_bound())

[40.661999  40.062999  10.4429997]


In [23]:
octree = o3d.geometry.Octree(max_depth=13,
                             origin=pcd_combined.get_center(),
                             size=0.01)
octree.convert_from_point_cloud(pcd_combined, size_expand=0.01)

In [28]:
p = pcds[0].points[0]
node = octree.locate_leaf_node(pcds[0].points[0])
node, node_info = octree.locate_leaf_node(p)
node_points = np.asarray(pcd_combined.points)[node.indices]

In [3]:
pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()),
          center=pcd.get_center())

PointCloud with 2502547 points.

In [5]:
# octree = o3d.geometry.Octree(max_depth=5)
octree = o3d.geometry.Octree(max_depth=5)
octree.convert_from_point_cloud(pcd, size_expand=0.01)

TypeError: convert_from_point_cloud(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cuda.pybind.geometry.Octree, point_cloud: open3d.cuda.pybind.geometry.PointCloud, size_expand: float = 0.01) -> None

Invoked with: Octree with origin: [0, 0, 0], size: 0, max_depth: 5, [PointCloud with 65 points., PointCloud with 107 points., PointCloud with 55 points., PointCloud with 113 points., PointCloud with 88 points., PointCloud with 72 points., PointCloud with 83 points., PointCloud with 84 points., PointCloud with 147 points., PointCloud with 135 points., PointCloud with 586 points., PointCloud with 57 points., PointCloud with 258 points., PointCloud with 91 points., PointCloud with 54 points., PointCloud with 91 points., PointCloud with 107 points., PointCloud with 86 points., PointCloud with 60 points., PointCloud with 63 points., PointCloud with 167 points., PointCloud with 294 points., PointCloud with 76 points., PointCloud with 69 poin

In [20]:
# pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(2000, 3)))
# o3d.visualization.draw_geometries([pcd])
# o3d.visualization.draw_geometries([octree])

In [32]:
node = octree.locate_leaf_node(point)
node

(OctreePointColorLeafNode with color [0.105882, 0.105882, 0.0784314] containing 2 points.,
 OctreeNodeInfo with origin [-50.3946, -8.67925, -1.32178], size 0.0315625, depth 5, child_index 3)

In [46]:
# 指定した点のボクセルを取得する関数
def find_voxel_for_point(octree, point):
    result = octree.locate_leaf_node(point)
    if result is not None:
        node, _ = result
        return node
    else:
        return None

# ノードの中心座標を取得する関数
def get_voxel_center(node, min_bound, voxel_size):
    center = min_bound + (np.asarray(node.origin) + voxel_size / 2.0)
    return center

# 近傍のボクセルを取得する関数
def find_neighbor_voxels(octree, node, min_bound, voxel_size):
    center = get_voxel_center(node, min_bound, voxel_size)
    neighbors = []
    directions = [
        np.array([1, 0, 0]), np.array([-1, 0, 0]),
        np.array([0, 1, 0]), np.array([0, -1, 0]),
        np.array([0, 0, 1]), np.array([0, 0, -1])
    ]
    
    for direction in directions:
        neighbor_center = center + direction * voxel_size
        result = octree.locate_leaf_node(neighbor_center)
        if result is not None:
            neighbor_node, _ = result
            neighbors.append(neighbor_node)
    
    return neighbors


point = pcd.points[0]
voxel = find_voxel_for_point(octree, point)

if voxel is not None:
    min_bound = pcd.get_min_bound()
    max_bound = pcd.get_max_bound()
    voxel_size = (max_bound - min_bound) / (2 ** octree.max_depth)
    
    neighbors = find_neighbor_voxels(octree, voxel, min_bound, voxel_size)
    print(f"Neighbors of the voxel containing {point}:")
    for neighbor in neighbors:
        print(get_voxel_center(neighbor, min_bound, voxel_size))
else:
    print(f"Point {point} is not in any voxel")

AttributeError: 'open3d.cuda.pybind.geometry.OctreePointColorLeafNo' object has no attribute 'origin'

In [61]:
point = pcd.points[0]
voxel = find_voxel_for_point(octree, point)
min_bound = pcd.get_min_bound()
max_bound = pcd.get_max_bound()
voxel_size = (max_bound - min_bound) / (2 ** octree.max_depth)

center = get_voxel_center(voxel, min_bound, voxel_size)

In [118]:
# voxel 内の点をすべて抽出
point = pcd.points[100]
node, node_info = octree.locate_leaf_node(point)
node_points = np.asarray(pcd.points)[node.indices]

neighbors = []
directions = [
    np.array([1, 0, 0]), np.array([-1, 0, 0]),
    np.array([0, 1, 0]), np.array([0, -1, 0]),
    np.array([0, 0, 1]), np.array([0, 0, -1])
]

for direction in directions:
    neighbor_center = node_info.origin + direction * node_info.size
    result = octree.locate_leaf_node(neighbor_center)
    if result is not None:
        neighbor_node, _ = result
        neighbors.append(neighbor_node)

print(neighbors)

[OctreePointColorLeafNode with color [0.211765, 0.219608, 0.2] containing 213 points., None, OctreePointColorLeafNode with color [0.133333, 0.129412, 0.113725] containing 302 points., OctreePointColorLeafNode with color [0.227451, 0.223529, 0.196078] containing 63 points., None, None]


In [119]:
octree

Octree with origin: [-50.4262, -8.83706, -1.76366], size: 1.01, max_depth: 5